<a href="https://colab.research.google.com/github/martinpius/Behind-Keras-Layers-and-Models/blob/main/keras_layers_and_models_on_the_background.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Setup the colab environment
from google.colab import drive
try:
  drive.mount("/content/drive", force_remount = True)
  COLAB = True
  import tensorflow as tf
  print(f"You are using google colab with tensorflow version: {tf.__version__}")
except Exception as e:
  COLAB = False
  print(f"{type(e)}: {e}\n----Import your google drive-----")
def timeset(x):
  hours = int(x/(60 *60))
  minutes = int(x % (60 *60)/60)
  seconds = int(x % 60)

Mounted at /content/drive
You are using google colab with tensorflow version: 2.3.1


In [2]:
pip install tensorflow --upgrade #Check for new version of tensorflow if available

Requirement already up-to-date: tensorflow in /usr/local/lib/python3.6/dist-packages (2.3.1)


In [3]:
import tensorflow as tf
print(f"Tensorflow Version: {tf.__version__}")

Tensorflow Version: 2.3.1


In [4]:
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import timeit
from datetime import datetime
plt.style.use("fivethirtyeight")


In [5]:
#We can easily convert tensorflow layers into keras layers by swtiching the argument (tf.Module with keras.layers.Layer)
#We also have to use call() inplace of __call__()..everything else remain similar

In [6]:
#A simple keras dense layer
class KLayer(keras.layers.Layer):
  def __init__(self, name = None):
    super(KLayer, self).__init__(name = name)
    self.dense_1 = keras.layers.Dense(units = 3, activation = 'relu')
  def call(self, inputs):
    out = self.dense_1(inputs)
    return out
klayer = KLayer(name = 'Test')
display(klayer(tf.constant([[2.,3.,2.]])))

<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[0.       , 3.3161464, 0.       ]], dtype=float32)>

In [7]:
#Altenative1 from scratch **Using add-weight
class KLayer1(keras.layers.Layer):
  def __init__(self, units = 32, input_shape = 28, name = None):
    super(KLayer1, self).__init__(name = name)
    self.w = self.add_weight(shape = (input_shape, units), initializer = 'random_normal', trainable = True)
    self.b = self.add_weight(shape = (units, ),initializer = 'zeros', trainable = True)
  def call(self, inputs):
    return tf.matmul(inputs, self.w) + self.b


x = tf.ones(shape = (10,3))
klayer1 = KLayer1(units = 3, input_shape = 3, name = 'test')
klayer(x)

<tf.Tensor: shape=(10, 3), dtype=float32, numpy=
array([[0.      , 1.268542, 0.      ],
       [0.      , 1.268542, 0.      ],
       [0.      , 1.268542, 0.      ],
       [0.      , 1.268542, 0.      ],
       [0.      , 1.268542, 0.      ],
       [0.      , 1.268542, 0.      ],
       [0.      , 1.268542, 0.      ],
       [0.      , 1.268542, 0.      ],
       [0.      , 1.268542, 0.      ],
       [0.      , 1.268542, 0.      ]], dtype=float32)>

In [8]:
#Altenative2 from scratch using tf.Variable to add weights
class KLayer2(keras.layers.Layer):
  def __init__(self, units = 32, input_shape = 128, **kwargs):
    super(KLayer2, self).__init__(**kwargs)
    self.w = tf.Variable(initial_value = tf.random.normal(shape = (input_shape, units)),trainable = True, name  = 'w')
    self.b = tf.Variable(initial_value = tf.zeros(shape = (units,)), trainable = True, name = 'b')

  def call(self, inputs):
    out = tf.matmul(inputs, self.w) + self.b
    return tf.nn.relu(out)

klayer2 = KLayer2(units = 3, input_shape = 3, name = 'test')
display(klayer2(tf.ones(shape = (3,3))))


<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[4.803815 , 2.6120744, 0.       ],
       [4.803815 , 2.6120744, 0.       ],
       [4.803815 , 2.6120744, 0.       ]], dtype=float32)>

In [9]:
#Difered the input shape in keras layer to add it later on the process
class KDiffer(keras.layers.Layer):
  def __init__(self, units, **kwargs):
    super(KDiffer, self).__init__(**kwargs)
    self.units = units
  def build(self, input_shape):
    self.w = tf.Variable(initial_value = tf.random.normal(shape = (input_shape[-1],self.units)),trainable = True, name = 'w')
    self.b = tf.Variable(initial_value = tf.zeros(shape = (self.units,)), trainable = True, name = 'b')
  def call(self, inputs):
    out = tf.matmul(inputs, self.w) + self.w
    return tf.nn.relu(out)

kdiffer = KDiffer(units = 3, name = 'test')
display(kdiffer(tf.ones(shape = (3,3))))

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[0.        , 0.5085881 , 0.        ],
       [0.        , 0.23611134, 0.        ],
       [0.        , 0.        , 0.        ]], dtype=float32)>

In [10]:
#Keras model is a version of keras layer with multiple additional functionality. Its literally
#inherit everything from keras layer.


In [51]:
#We can produce a simple keras sequential model from the above defined layer
class KModel(keras.Model):
  def __init__(self, **kwargs):
    super(KModel, self).__init__(**kwargs)
    self.l1 = KDiffer(units = 128, name = 'layer_1')
    self.l2 = KDiffer(units = 64, name = 'layer_2')
    

  def call(self, inputs):
    inputs = self.l1(inputs)
    inputs = self.l2(inputs)
    return tf.nn.softmax(inputs)
model_test = KModel(name = 'Sequential_test')
x = tf.ones(shape = (128,128))

In [52]:
model_test(x)

<tf.Tensor: shape=(128, 64), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>

In [72]:
#Get the same keras model using functional API (But we have already create our customized layers)
inputs = keras.Input(shape = [32,])
x = KDiffer(units = 128, name = 'layer_1')(inputs)
x = KDiffer(units = 64, name = 'layer_2')(inputs)
out = tf.nn.softmax(x)
model = keras.Model(inputs = inputs,outputs = out)

In [73]:
model.summary()

Model: "functional_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 32)]              0         
_________________________________________________________________
layer_2 (KDiffer)            (32, 64)                  2112      
_________________________________________________________________
tf_op_layer_Softmax_3 (Tenso [(32, 64)]                0         
Total params: 2,112
Trainable params: 2,112
Non-trainable params: 0
_________________________________________________________________


In [74]:
model(tf.ones(shape = [32,32], dtype= tf.float32))

<tf.Tensor: shape=(32, 64), dtype=float32, numpy=
array([[8.24821313e-08, 8.24821313e-08, 8.24821313e-08, ...,
        8.24821313e-08, 8.24821313e-08, 8.24821313e-08],
       [3.39552351e-08, 3.39552351e-08, 3.39552351e-08, ...,
        3.39552351e-08, 3.39552351e-08, 3.39552351e-08],
       [6.83391077e-09, 6.83391077e-09, 6.83391077e-09, ...,
        6.83391077e-09, 6.83391077e-09, 6.83391077e-09],
       ...,
       [4.81592188e-08, 4.81592188e-08, 4.81592188e-08, ...,
        4.81592188e-08, 4.81592188e-08, 4.81592188e-08],
       [1.49197064e-07, 1.49197064e-07, 1.49197064e-07, ...,
        1.49197064e-07, 1.49197064e-07, 1.49197064e-07],
       [1.04471056e-07, 1.04471056e-07, 1.04471056e-07, ...,
        1.04471056e-07, 1.04471056e-07, 1.04471056e-07]], dtype=float32)>

In [75]:
#Save and re-load keras models 
model.save('mysample_model')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: mysample_model/assets


In [77]:
new_model = keras.models.load_model('mysample_model')

In [79]:
#Test if it works as expected
new_model(tf.ones(shape = (32,32)))

<tf.Tensor: shape=(32, 64), dtype=float32, numpy=
array([[8.24821313e-08, 8.24821313e-08, 8.24821313e-08, ...,
        8.24821313e-08, 8.24821313e-08, 8.24821313e-08],
       [3.39552351e-08, 3.39552351e-08, 3.39552351e-08, ...,
        3.39552351e-08, 3.39552351e-08, 3.39552351e-08],
       [6.83391077e-09, 6.83391077e-09, 6.83391077e-09, ...,
        6.83391077e-09, 6.83391077e-09, 6.83391077e-09],
       ...,
       [4.81592188e-08, 4.81592188e-08, 4.81592188e-08, ...,
        4.81592188e-08, 4.81592188e-08, 4.81592188e-08],
       [1.49197064e-07, 1.49197064e-07, 1.49197064e-07, ...,
        1.49197064e-07, 1.49197064e-07, 1.49197064e-07],
       [1.04471056e-07, 1.04471056e-07, 1.04471056e-07, ...,
        1.04471056e-07, 1.04471056e-07, 1.04471056e-07]], dtype=float32)>